# <ins>Build up a composite model: Radial symmetry</ins>

## Core-shell system

Let's consider a spherical colloid with a different density or chemical composition along the radial direction. The scattering amplitude will then be:

$A(q)= \ 4\pi \ \int_0^R r^2 \Delta \rho(r) \ j_0(qr) \ \text{d}r$

The simplest case is the core-shell system: *1)* a core compartment of SLD $\rho_0$ and radius $R_0$, *2)* a shell of SLD $\rho_1$ and thickness $\Delta R$.

<img src="./imgs/core_shell_0.png" alt="Core-shell sketch" width="700"/>

There are at least 2 ways to solve this integral. These can be ***visualized*** as such

<img src="./imgs/core_shell_1.png" alt="Core-shell sketch 2" width="1600"/>



In [1]:
#!/usr/bin/python

import math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from ipywidgets import interact,FloatSlider

import library.Sphere_models as sph

fs = 9

rho_scale = np.linspace(7.0*1e-4, 16.0*1e-4, 50, endpoint=True)

def normalize(x, rmax=rho_scale[-1], rmin=rho_scale[0]):
    return (x-rmax)/(rmin-rmax)

def vol(r):
    return 4*math.pi*r**3/3

# Concentration
n = 1e-10 # nm^-3

In [2]:
def interactive_shell( R, rel, DR, r0, r1, rs ):
    # dynamicaly initialize q-array (3 orders of magnitude only)
    min_order = math.log(0.5/(R+DR), 10)
    q = np.logspace(min_order,min_order+3, num=500, endpoint=True, base=10.0)
    
    # Initialize plot
    fig, axs = plt.subplots(1,2,figsize=(fs*2,fs*0.7))
    
    axs[0].set_xlabel('$q$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_ylabel('$I(q)$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_xscale('log')
    axs[0].set_yscale('log')
    axs[0].minorticks_on()
    axs[0].tick_params(axis='both', which='major', labelsize=14)
    
    rho_ave = ( vol(R)*r0 + (vol(R+DR)-vol(R))*r1 ) / (vol(R+DR))

    # plot I(q) curves
    q_max = 10. # nm^-1
    I_eq_sphere = sph.sphere_int_normal(q,n,R+DR,(R+DR)*rel,(rho_ave-rs))
    axs[0].plot(q, np.where(q<=q_max,I_eq_sphere,np.nan), color="gray", label='Equivalent homogeneous sphere')
    I_shell = sph.sphere_core_shell_int_normal(q,n,R,R*rel,DR,rs,r0,r1)
    axs[0].plot(q, np.where(q<=q_max,I_shell,np.nan), color="red", label='Core-shell system')

    # plot vertical lines
    axs[0].axvline(x=2*math.pi/DR,color="blue",ls="--", label='$2\\pi/\\Delta R$')
    axs[0].axvline(x=4.5/(R+DR),color="red",ls=":", label='$4.5/(R+\\Delta R)$')

    # include WAXS regime
    #bottom = np.min([np.min(I_eq_sphere),np.min(I_shell)])
    #top = np.max([np.max(I_eq_sphere),np.max(I_shell)])
    #right = np.max([10,np.max(q)])
    #axs[0].add_patch( plt.Rectangle((q_max, bottom), right-q_max, top-bottom, facecolor="black", alpha=0.2, zorder=2) )
    #if np.max(q)>=10: axs[0].text(q_max+1, (top-bottom)*1e-4, 'WAXS regime', rotation=-90)

    axs[0].legend()

    c0 = plt.Circle((0, 0), radius=R,    color=str(normalize(r0)), label='0')
    c1 = plt.Circle((0, 0), radius=R+DR, color=str(normalize(r1)), label='1')
    
    axs[1].add_patch(c1)
    axs[1].add_patch(c0)
    axs[1].set_facecolor(str(normalize(rs)))

    cmap = matplotlib.cm.gray
    norm = matplotlib.colors.Normalize(vmin=rho_scale[0], vmax=rho_scale[-1])

    fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axs[1], orientation='vertical', label='$\\rho$ (nm$^{-2}$)')
    
    plt.gca().relim()  # Recalculate limits
    plt.gca().autoscale_view()  # Auto-scale the view
    plt.show()

style = {'description_width': 'initial'}

interact(interactive_shell,  R = FloatSlider(value=40, min=1, max=100, step=1, readout_format='.0f', description='Core radius (nm)',style=style),
                             rel = FloatSlider(value=0.1, min=0.02, max=0.24, step=0.02, readout_format='.2p', description='PDI',style=style),
                             DR = FloatSlider(value=5., min=0.5, max=50, step=0.5, readout_format='.1f', description='Shell width (nm)',style=style),
                             r0 = FloatSlider(value=16.*1e-4, min=rho_scale[0], max=rho_scale[-1], step=rho_scale[1]-rho_scale[0], readout_format='.2e', description='Core SLD (nm$^{-2}$)',style=style),
                             r1 = FloatSlider(value=11.*1e-4, min=rho_scale[0], max=rho_scale[-1], step=rho_scale[1]-rho_scale[0], readout_format='.2e', description='Shell SLD (nm$^{-2}$)',style=style),  
                             rs = FloatSlider(value=9.4*1e-4, min=rho_scale[0], max=rho_scale[-1], step=rho_scale[1]-rho_scale[0], readout_format='.2e', description='Medium SLD (nm$^{-2}$)',style=style))

interactive(children=(FloatSlider(value=40.0, description='Core radius (nm)', min=1.0, readout_format='.0f', s…

<function __main__.interactive_shell(R, rel, DR, r0, r1, rs)>

## Large Unilamellar Vesicle - SAXS example

In [3]:
#initial values
R_0=46 # nm
rel_0=0.1
d_h_0=0.5
d_t_0=1.0
d_m_0=0.1
r_h_0=12.*1e-4 # nm-2
r_t_0=8.*1e-4 # nm-2
r_m_0=7.*1e-4 # nm-2
r_s_0=9.4*1e-4 # nm-2

# length lipid head
d_h_min = 0.3
d_h_max = 0.9
# length lipid tail
d_t_min = 0.8
d_t_max = 2.0
# length methyl
d_m_min = 0.
d_m_max = 0.2
# SLD lipid head
rho_h_saxs = [11.0*1e-4, 14.0*1e-4] # nm-2
# SLD lipid tail
rho_t_saxs = [7.2*1e-4, 8.6*1e-4] # nm-2
# SLD methyl
rho_m_saxs = [6.6*1e-4, 7.2*1e-4] # nm-2
# SLD solvent
rho_s_saxs = [9.2*1e-4, 10.0*1e-4] # nm-2

def interactive_shell( R, rel, d_h, d_t, d_m, r_h, r_t, r_m, r_s ):
    # dynamicaly initialize q-array (3 orders of magnitude only)
    min_order = math.log(0.5/(R+2*d_h+2*d_t), 10)
    q = np.logspace(min_order,min_order+3, num=500, endpoint=True, base=10.0)

    # Initialize plot
    fig, axs = plt.subplots(1,2,figsize=(fs*2,fs*0.7))
    
    axs[0].set_xlabel('$q$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_ylabel('$I(q)$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_xscale('log')
    axs[0].set_yscale('log')
    axs[0].minorticks_on()
    axs[0].tick_params(axis='both', which='major', labelsize=14)

    V_core  = vol(R)
    V_h_in  = vol(R+d_h) - vol(R)
    V_t_in  = vol(R+d_h+d_t) - vol(R+d_h)
    V_m     = vol(R+d_h+d_t+d_m) - vol(R+d_h+d_t)
    V_t_out = vol(R+d_h+2*d_t+d_m) - vol(R+d_h+d_t+d_m)
    V_h_out = vol(R+2*d_h+2*d_t+d_m) - vol(R+d_h+2*d_t+d_m)
    V_h = V_h_in + V_h_out
    V_t = V_t_in + V_t_out

    rho_ave = ( V_core*r_s + V_h*r_h + V_t*r_t + V_m*r_m ) / (V_core+V_h+V_t+V_m)
    
    # plot I(q) curves
    q_max = 10. # nm^-1
    I_shell_0 = sph.sphere_core_5xshell_int_normal (q,n,R_0,R_0*rel_0, d_h_0,d_t_0,d_m_0, r_h_0,r_t_0,r_m_0,r_s_0)
    axs[0].plot(q, np.where(q<=q_max,I_shell_0,np.nan), color="gray", label='Reference')
    I_shell = sph.sphere_core_5xshell_int_normal (q,n,R,R*rel,d_h,d_t,d_m,r_h,r_t,r_m,r_s)
    axs[0].plot(q, np.where(q<=q_max,I_shell,np.nan), color="red", label='Core-(5x)shells system: Liposome')

    axs[0].axvline(2*math.pi/(d_h+d_t+d_m+d_t+d_h),color="blue",ls="--", label='$2\\pi/\\Delta R$')

    axs[0].legend()

    c0 = plt.Circle((0, 0), radius=R,                     color=str(normalize(r_s, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='0')
    c1 = plt.Circle((0, 0), radius=R+d_h,                 color=str(normalize(r_h, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='1')
    c2 = plt.Circle((0, 0), radius=R+d_h+d_t,             color=str(normalize(r_t, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='2')
    c3 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m,         color=str(normalize(r_m, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='3')
    c4 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m+d_t,     color=str(normalize(r_t, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='4')
    c5 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m+d_t+d_h, color=str(normalize(r_h, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])), label='5')

    axs[1].add_patch(c5)
    axs[1].add_patch(c4)
    axs[1].add_patch(c3)
    axs[1].add_patch(c2)
    axs[1].add_patch(c1)
    axs[1].add_patch(c0)
    axs[1].set_facecolor(str(normalize(r_s, rmax=rho_h_saxs[1], rmin=rho_m_saxs[0])))

    cmap = matplotlib.cm.gray
    norm = matplotlib.colors.Normalize(vmin=rho_m_saxs[0], vmax=rho_h_saxs[1])

    fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axs[1], orientation='vertical', label='$\\rho$ (nm$^{-2}$)')
    
    plt.gca().relim()  # Recalculate limits
    plt.gca().autoscale_view()  # Auto-scale the view
    plt.show()

style = {'description_width': 'initial'}

interact(interactive_shell,  R = FloatSlider(value=R_0, min=12, max=200, step=2, readout_format='.0f', description='Core radius (nm)',style=style),
                             rel = FloatSlider(value=rel_0, min=0.02, max=0.20, step=0.02, readout_format='.2p', description='PDI',style=style),
                             d_h = FloatSlider(value=d_h_0, min=d_h_min, max=d_h_max, step=0.1, readout_format='.1f', description='Lipid head width (nm)',style=style),
                             d_t = FloatSlider(value=d_t_0, min=d_t_min, max=d_t_max, step=0.1, readout_format='.1f', description='Lipid tail width (nm)',style=style),
                             d_m = FloatSlider(value=d_m_0, min=d_m_min, max=d_m_max, step=0.05, readout_format='.2f', description='Lipi methyl width (nm)',style=style),
                             r_h = FloatSlider(value=r_h_0, min=rho_h_saxs[0], max=rho_h_saxs[1], step=0.2*1e-4, readout_format='.2e', description='Lipid head SLD (nm$^{-2}$)',style=style),
                             r_t = FloatSlider(value=r_t_0, min=rho_t_saxs[0], max=rho_t_saxs[1], step=0.2*1e-4, readout_format='.2e', description='Lipid tail SLD (nm$^{-2}$)',style=style),
                             r_m = FloatSlider(value=r_m_0, min=rho_m_saxs[0], max=rho_m_saxs[1], step=0.2*1e-4, readout_format='.2e', description='Methyl SLD (nm$^{-2}$)',style=style),  
                             r_s = FloatSlider(value=r_s_0, min=rho_s_saxs[0], max=rho_s_saxs[1], step=0.2*1e-4, readout_format='.2e', description='Medium SLD (nm$^{-2}$)',style=style))


interactive(children=(FloatSlider(value=46.0, description='Core radius (nm)', max=200.0, min=12.0, readout_for…

<function __main__.interactive_shell(R, rel, d_h, d_t, d_m, r_h, r_t, r_m, r_s)>

## Large Unilamellar Vesicle - SANS example

In [4]:
#initial values
R_0=46 # nm
rel_0=0.1
d_h_0=0.5
d_t_0=1.0
d_m_0=0.1
r_h_0=1.8*1e-4 # nm-2
r_t_0=-0.2*1e-4 # nm-2
r_m_0=-0.2*1e-4 # nm-2
r_s_0=-5.5*1e-4 # nm-2

# length lipid head
d_h_min = 0.3
d_h_max = 0.9
# length lipid tail
d_t_min = 0.8
d_t_max = 2.0
# length methyl
d_m_min = 0.
d_m_max = 0.2
# SLD lipid head
rho_h_sans = [-0.6*1e-4, 2.2*1e-4] # nm-2
# SLD lipid tail
rho_t_sans = [-1.*1e-4, +3.*1e-4] # nm-2
# SLD methyl
rho_m_sans = [-1.*1e-4, +3.*1e-4] # nm-2
# SLD solvent
rho_s_sans = [-5.5*1e-4, +6.36*1e-4] # nm-2




def interactive_shell( R, rel, d_h, d_t, d_m, r_h, r_t, r_m, r_s ):
    # dynamicaly initialize q-array (3 orders of magnitude only)
    min_order = math.log(0.5/(R+2*d_h+2*d_t), 10)
    q = np.logspace(min_order,min_order+3, num=500, endpoint=True, base=10.0)
    
    # Initialize plot
    fig, axs = plt.subplots(1,2,figsize=(fs*2,fs*0.7))
    
    axs[0].set_xlabel('$q$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_ylabel('$I(q)$ (nm$^{-1}$)', fontsize=16)
    axs[0].set_xscale('log')
    axs[0].set_yscale('log')
    axs[0].minorticks_on()
    axs[0].tick_params(axis='both', which='major', labelsize=14)

    V_core  = vol(R)
    V_h_in  = vol(R+d_h) - vol(R)
    V_t_in  = vol(R+d_h+d_t) - vol(R+d_h)
    V_m     = vol(R+d_h+d_t+d_m) - vol(R+d_h+d_t)
    V_t_out = vol(R+d_h+2*d_t+d_m) - vol(R+d_h+d_t+d_m)
    V_h_out = vol(R+2*d_h+2*d_t+d_m) - vol(R+d_h+2*d_t+d_m)
    V_h = V_h_in + V_h_out
    V_t = V_t_in + V_t_out

    rho_ave = ( V_core*r_s + V_h*r_h + V_t*r_t + V_m*r_m ) / (V_core+V_h+V_t+V_m)

    # plot I(q) curves
    q_max = 10. # nm^-1
    I_shell_0 = sph.sphere_core_5xshell_int_normal (q,n,R_0,R_0*rel_0, d_h_0,d_t_0,d_m_0, r_h_0,r_t_0,r_m_0,r_s_0)
    axs[0].plot(q, np.where(q<=q_max,I_shell_0,np.nan), color="gray", label='Reference')
    I_shell = sph.sphere_core_5xshell_int_normal (q,n,R,R*rel,d_h,d_t,d_m,r_h,r_t,r_m,r_s)
    axs[0].plot(q, np.where(q<=q_max,I_shell,np.nan), color="red", label='Core-(5x)shells system: Liposome')

    axs[0].axvline(2*math.pi/(d_h+d_t+d_m+d_t+d_h),color="blue",ls="--", label='$2\\pi/\\Delta R$')

    axs[0].legend()

    c0 = plt.Circle((0, 0), radius=R,                     color=str(normalize(r_s, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='0')
    c1 = plt.Circle((0, 0), radius=R+d_h,                 color=str(normalize(r_h, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='1')
    c2 = plt.Circle((0, 0), radius=R+d_h+d_t,             color=str(normalize(r_t, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='2')
    c3 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m,         color=str(normalize(r_m, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='3')
    c4 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m+d_t,     color=str(normalize(r_t, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='4')
    c5 = plt.Circle((0, 0), radius=R+d_h+d_t+d_m+d_t+d_h, color=str(normalize(r_h, rmax=rho_s_sans[1], rmin=rho_s_sans[0])), label='5')

    axs[1].add_patch(c5)
    axs[1].add_patch(c4)
    axs[1].add_patch(c3)
    axs[1].add_patch(c2)
    axs[1].add_patch(c1)
    axs[1].add_patch(c0)
    axs[1].set_facecolor(str(normalize(r_s, rmax=rho_s_sans[1], rmin=rho_s_sans[0])))

    cmap = matplotlib.cm.gray
    norm = matplotlib.colors.Normalize(vmin=rho_s_sans[0], vmax=rho_s_sans[1])

    fig.colorbar(matplotlib.cm.ScalarMappable(norm=norm, cmap=cmap), ax=axs[1], orientation='vertical', label='$\\rho$ (nm$^{-2}$)')
    
    plt.gca().relim()  # Recalculate limits
    plt.gca().autoscale_view()  # Auto-scale the view
    plt.show()


style = {'description_width': 'initial'}

interact(interactive_shell,  R = FloatSlider(value=R_0, min=12, max=200, step=2, readout_format='.0f', description='Core radius (nm)',style=style),
                             rel = FloatSlider(value=rel_0, min=0.02, max=0.20, step=0.02, readout_format='.2p', description='PDI',style=style),
                             d_h = FloatSlider(value=d_h_0, min=d_h_min, max=d_h_max, step=0.1, readout_format='.1f', description='Lipid head width (nm)',style=style),
                             d_t = FloatSlider(value=d_t_0, min=d_t_min, max=d_t_max, step=0.1, readout_format='.1f', description='Lipid tail width (nm)',style=style),
                             d_m = FloatSlider(value=d_m_0, min=d_m_min, max=d_m_max, step=0.05, readout_format='.2f', description='Lipi methyl width (nm)',style=style),
                             r_h = FloatSlider(value=r_h_0, min=rho_h_sans[0], max=rho_h_sans[1], step=0.2*1e-4, readout_format='.2e', description='Lipid head SLD (nm$^{-2}$)',style=style),
                             r_t = FloatSlider(value=r_t_0, min=rho_t_sans[0], max=rho_t_sans[1], step=0.2*1e-4, readout_format='.2e', description='Lipid tail SLD (nm$^{-2}$)',style=style),
                             r_m = FloatSlider(value=r_m_0, min=rho_m_sans[0], max=rho_m_sans[1], step=0.2*1e-4, readout_format='.2e', description='Methyl SLD (nm$^{-2}$)',style=style),  
                             r_s = FloatSlider(value=r_s_0, min=rho_s_sans[0], max=rho_s_sans[1], step=0.2*1e-4, readout_format='.2e', description='Medium SLD (nm$^{-2}$)',style=style))


interactive(children=(FloatSlider(value=46.0, description='Core radius (nm)', max=200.0, min=12.0, readout_for…

<function __main__.interactive_shell(R, rel, d_h, d_t, d_m, r_h, r_t, r_m, r_s)>